In [3]:
#### CRITICAL - ENABLE GPU 

import pandas as pd
import numpy as np
import json, re
from tqdm import tqdm_notebook
from uuid import uuid4

## Torch Modules
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader


## Mount Drive into Colab
#from google.colab import drive
#drive.mount('/content/drive')

In [4]:
!pip install pytorch-transformers

Defaulting to user installation because normal site-packages is not writeable


In [5]:
## PyTorch Transformer
from pytorch_transformers import RobertaModel, RobertaTokenizer
from pytorch_transformers import RobertaForSequenceClassification, RobertaConfig

In [6]:
## Check if Cuda is Available
print(torch.cuda.is_available())

True


## Install PyTorch-Transformer

In [7]:
!pip install -U pytorch-transformers

Defaulting to user installation because normal site-packages is not writeable


## Importing Datasets

In [8]:
'''
Important Step - Make sure you upload the data file to the exact location below. If you uploaded correctlt, the following command will run
'''

! ls 2017-06-custom-intent-engines

AddToPlaylist	GetWeather  RateBook   SearchCreativeWork
BookRestaurant	PlayMusic   README.md  SearchScreeningEvent


In [9]:
'''
Create the Dataset Path
'''


dataset_path = "2017-06-custom-intent-engines/"

In [10]:

'''
***Explain*** Summarize, in bullet points, what is the code doing?. 
'''

dataset = pd.DataFrame(columns = ['utterance', 'label'])
for intent in ['AddToPlaylist', 'BookRestaurant', 'GetWeather', 'PlayMusic', 'RateBook', 'SearchCreativeWork',
               'SearchScreeningEvent']:
    with open(dataset_path + intent + "/train_" + intent + ".json",
              encoding='cp1251') as data_file:
        data = json.load(data_file)
    print("Class: {}, # utterances: {}".format(intent,len(data[intent])))
    texts = []
    for i in range(len(data[intent])):
        text = ''
        for j in range(len(data[intent][i]['data'])):
            text += data[intent][i]['data'][j]['text']
        dataset = pd.concat([dataset, pd.DataFrame([{'utterance': text, 'label': intent}])], ignore_index=True)
dataset.tail()

Class: AddToPlaylist, # utterances: 300
Class: BookRestaurant, # utterances: 300
Class: GetWeather, # utterances: 300
Class: PlayMusic, # utterances: 300
Class: RateBook, # utterances: 300
Class: SearchCreativeWork, # utterances: 300
Class: SearchScreeningEvent, # utterances: 300


,utterance,label
2095,Is Across the Line playing at the closest movi...,SearchScreeningEvent
2096,Which animated movies are playing in the neigh...,SearchScreeningEvent
2097,Where is They Always Return at Dawn playing,SearchScreeningEvent
2098,What is the movie schedule in the neighborhood,SearchScreeningEvent
2099,Tell me when Howling II: Your Sister Is a Were...,SearchScreeningEvent


In [11]:
'''
Assigning an Index to each intent. We will use this later
'''

'''
***Explain*** Why do we convert labels to indexes?. 
'''

label_to_ix = {}
for label in dataset.label:
    for word in label.split():
        if word not in label_to_ix:
            label_to_ix[word]=len(label_to_ix)
label_to_ix

{'AddToPlaylist': 0,
 'BookRestaurant': 1,
 'GetWeather': 2,
 'PlayMusic': 3,
 'RateBook': 4,
 'SearchCreativeWork': 5,
 'SearchScreeningEvent': 6}

## Loading RoBERTa classes

In [12]:
config = RobertaConfig.from_pretrained('roberta-base')
config.num_labels = len(list(label_to_ix.values()))
config

{
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 7,
  "output_attentions": false,
  "output_hidden_states": false,
  "pad_token_id": 1,
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_size": 1,
  "vocab_size": 50265
}

In [13]:
'''
Loading Pretrained tokenizer and instantiating the model from settings in config
'''

'''
***Explain*** : a. What is a tokenizer? b. What is special about the following tokenizer?. 
'''

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

'''
***Explain*** :  What is the next line doing?
'''

model = RobertaForSequenceClassification(config)

## Feature Preparation

In [14]:
'''
Some important Feature Engineering
'''

'''
***Explain*** : What are the implications for setting  include_CLS_token = True, include_SEP_token = True ?
'''

def prepare_features(seq_1, max_seq_length = 300, 
             zero_pad = False, include_CLS_token = True, include_SEP_token = True):
    ## Tokenzine Input
    tokens_a = tokenizer.tokenize(seq_1)

    ## Truncate
    if len(tokens_a) > max_seq_length - 2:
        tokens_a = tokens_a[0:(max_seq_length - 2)]
    ## Initialize Tokens
    tokens = []
    if include_CLS_token:
        tokens.append(tokenizer.cls_token)
    ## Add Tokens and separators
    for token in tokens_a:
        tokens.append(token)

    if include_SEP_token:
        tokens.append(tokenizer.sep_token)

    input_ids = tokenizer.convert_tokens_to_ids(tokens)
    ## Input Mask 
    input_mask = [1] * len(input_ids)
    ## Zero-pad sequence lenght
    if zero_pad:
        while len(input_ids) < max_seq_length:
            input_ids.append(0)
            input_mask.append(0)
    return torch.tensor(input_ids).unsqueeze(0), input_mask

In [15]:
msg = "My dog is cute!"
prepare_features(msg)

(tensor([[    0,  1308,  2335,    16, 11962,   328,     2]]),
 [1, 1, 1, 1, 1, 1, 1])

## Dataset Loader Classes

In [16]:
class Intents(Dataset):
    def __init__(self, dataframe):
        self.len = len(dataframe)
        self.data = dataframe
        
    def __getitem__(self, index):
        utterance = self.data.utterance[index]
        label = self.data.label[index]
        X, _  = prepare_features(utterance)
        y = label_to_ix[self.data.label[index]]
        return X, y
    
    def __len__(self):
        return self.len

In [17]:
train_size = 0.8
train_dataset=dataset.sample(frac=train_size,random_state=200).reset_index(drop=True)
test_dataset=dataset.drop(train_dataset.index).reset_index(drop=True)

In [18]:
print("FULL Dataset: {}".format(dataset.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

FULL Dataset: (2100, 2)
TRAIN Dataset: (1680, 2)
TEST Dataset: (420, 2)


In [19]:
training_set = Intents(train_dataset)
testing_set = Intents(test_dataset)

In [20]:
training_set.__getitem__(0)[0].shape

torch.Size([1, 8])

In [21]:
model(training_set.__getitem__(0)[0])

(tensor([[ 1.2699e-01, -1.7851e-01,  7.2745e-02, -4.3752e-05,  6.3752e-03,
          -1.7097e-01,  2.1368e-01]], grad_fn=<AddmmBackward0>),)

## Training Params

In [22]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.cuda()

In [23]:
# Parameters
params = {'batch_size': 1,
          'shuffle': True,
          'drop_last': False,
          'num_workers': 1
          }

In [24]:
training_loader = DataLoader(training_set, **params)
testing_loader = DataLoader(testing_set, **params)

In [25]:
'''
Instantiate the Loss
'''
'''
***Explain*** why cross entropy loss?, also print the model and explain why are not we using softmax at the end?
'''
loss_function = nn.CrossEntropyLoss()
learning_rate = 1e-05
optimizer = optim.Adam(params =  model.parameters(), lr=learning_rate)
print(model)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=0)
      (position_embeddings): Embedding(514, 768)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,)

In [30]:
## Test Forward Pass
inp = training_set.__getitem__(0)[0].cuda()
output = model(inp)[0]
print(output.shape)

torch.Size([1, 7])


In [31]:
torch.__version__

'2.0.1+cu118'

In [26]:
'''
Actually train the model with train data
'''
'''
***Explain*** the Training Code Chunk in detail. Especially what is torch.max() doing here?
'''



max_epochs = 3
model = model.train()
for epoch in tqdm_notebook(range(max_epochs)):
    print("EPOCH -- {}".format(epoch))
    for i, (sent, label) in enumerate(training_loader):
        optimizer.zero_grad()
        sent = sent.squeeze(0)
        if torch.cuda.is_available():
          sent = sent.cuda()
          label = label.cuda()
        output = model.forward(sent)[0]
        _, predicted = torch.max(output, 1)
        
        loss = loss_function(output, label)
        loss.backward()
        optimizer.step()
        
        if i%100 == 0:
            correct = 0
            total = 0
            for sent, label in testing_loader:
                sent = sent.squeeze(0)
                if torch.cuda.is_available():
                  sent = sent.cuda()
                  label = label.cuda()
                output = model.forward(sent)[0]
                _, predicted = torch.max(output.data, 1)
                total += label.size(0)
                correct += (predicted.cpu() == label.cpu()).sum()
            accuracy = 100.00 * correct.numpy() / total
            print('Iteration: {}. Loss: {}. Accuracy: {}%'.format(i, loss.item(), accuracy))

/tmp/ipykernel_19050/1510112102.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for epoch in tqdm_notebook(range(max_epochs)):


  0%|          | 0/3 [00:00<?, ?it/s]

EPOCH -- 0
Iteration: 0. Loss: 1.5319921970367432. Accuracy: 71.42857142857143%
Iteration: 100. Loss: 1.5659877061843872. Accuracy: 0.0%
Iteration: 200. Loss: 1.8033266067504883. Accuracy: 67.61904761904762%
Iteration: 300. Loss: 1.6214380264282227. Accuracy: 3.5714285714285716%
Iteration: 400. Loss: 2.1860790252685547. Accuracy: 28.571428571428573%
Iteration: 500. Loss: 1.1266999244689941. Accuracy: 17.38095238095238%
Iteration: 600. Loss: 1.0576212406158447. Accuracy: 15.952380952380953%
Iteration: 700. Loss: 1.2241374254226685. Accuracy: 38.80952380952381%
Iteration: 800. Loss: 0.6245338916778564. Accuracy: 56.904761904761905%
Iteration: 900. Loss: 0.27856847643852234. Accuracy: 50.0%
Iteration: 1000. Loss: 0.016908885911107063. Accuracy: 33.333333333333336%
Iteration: 1100. Loss: 0.060578059405088425. Accuracy: 93.33333333333333%
Iteration: 1200. Loss: 0.8997507095336914. Accuracy: 79.04761904761905%
Iteration: 1300. Loss: 0.049345821142196655. Accuracy: 71.42857142857143%
Iteratio

In [27]:
'''
***Explain*** what is the get_reply function doing?
'''
def get_reply(msg):
  model.eval()
  input_msg, _ = prepare_features(msg)
  if torch.cuda.is_available():
    input_msg = input_msg.cuda()
  output = model(input_msg)[0]
  _, pred_label = torch.max(output.data, 1)
  prediction=list(label_to_ix.keys())[pred_label]
  return prediction

In [28]:
label_to_ix.keys()

dict_keys(['AddToPlaylist', 'BookRestaurant', 'GetWeather', 'PlayMusic', 'RateBook', 'SearchCreativeWork', 'SearchScreeningEvent'])

In [29]:
'''Different text sentences pass to the model'''

get_reply("play radiohead song")

'PlayMusic'

In [30]:
get_reply("it is rainy in Sao Paulo")

'GetWeather'

In [36]:
get_reply("sun shinnes all day")

'PlayMusic'

In [32]:
get_reply("low humidity, high altitude")

'SearchCreativeWork'

In [33]:
get_reply("Book tacos for me tonight")

'BookRestaurant'

In [34]:
get_reply("Book a table for me tonight")

'BookRestaurant'

In [35]:
get_reply("I want BBQ tonight")

'PlayMusic'

In [37]:
print(get_reply('Who wrote the book The Martian?'))
print(get_reply('Book a me a table at Song Bird'))
print(get_reply('What is the weather in Boston?'))
print(get_reply('Add shine on you crazy diamond to my playlist'))
print(get_reply('Is where the red fern grows a good book?'))

SearchCreativeWork
BookRestaurant
GetWeather
AddToPlaylist
SearchCreativeWork
